In [1]:
PROCESSED_DATA_FP = '../osm-data/processed/'
MAP_FP = PROCESSED_DATA_FP + 'maps/'

In [2]:
import json
import os
import fiona
BASE_DIR = os.path.dirname(os.getcwd())
os.chdir(BASE_DIR + '/src/data/')
import util
os.chdir(BASE_DIR + '/notebooks/')

In [3]:
items = json.load(open(PROCESSED_DATA_FP + 'crash_joined.json'))
crash_list, crashes_by_id = util.group_json_by_location(items)
print str(len(crash_list)) + " crashes found"


# Crashes by id is a dict where the key is the segment id.  It contains the count of crashes at that intersection
# Can also contain other information about this location, if called with arg otherfields.  In this case, we're
# Looking at time
sorted = [value['count'] for key,value in crashes_by_id.items() if key != '']
sorted.sort(reverse=True)
print str(len(sorted)) + " unique crash locations found\n"

print "Highest number of crashes at any one location: " + str(sorted[0:10])
worst_id = [key for key, value in crashes_by_id.items() if value['count'] == sorted[0]][0]


7220 crashes found
3379 unique crash locations found

Highest number of crashes at any one location: [38, 34, 31, 29, 29, 27, 27, 27, 22, 21]


In [4]:
def get_int_feat(featname, items, buckets=[]):
    results = {}
    for id, values in items.iteritems():
        value = max([x[featname] for x in values])
        
        if buckets:
            found = False
            for bucket in buckets:
                if int(value) <= bucket and not found:
                    value = bucket
                    found = True
            if not found:
                value = buckets[len(buckets)-1]
        if value not in results.keys():
            results[value] = {'crash': 0, 'no_crash': 0}
        if id in crashes_by_id.keys():
            results[value]['crash'] += 1
        else:
            results[value]['no_crash'] += 1
 
    print "\n" + featname + ".............."
    for key, value in results.iteritems():
        print str(key) + ':' + str(float(value['crash'])/(float(value['no_crash'] + value['crash']))) + " out of " + str(value['no_crash'] + value['crash'])


In [5]:
items = json.load(open(PROCESSED_DATA_FP + '/inters_data.json'))

#feats = ['hwy_type', 'width', 'lanes', 'osm_speed', 'signal', 'SPEEDLIMIT', 'Struct_Cnd', 'Surface_Tp', 'F_F_Class']

print len(items)
print "************************************"
get_int_feat('signal', items)
get_int_feat('lanes', items)
get_int_feat('width', items, [9, 14, 19, 29])

# Highway types are kind of problematic to use max for, but it looks like if they're all residential roads, that really reduces crash risk
# Also, too many highway types with combo types (e.g. motorway and motorway link...)
get_int_feat('hwy_type', items)

get_int_feat('SPEEDLIMIT', items)

get_int_feat('Struct_Cnd', items)

get_int_feat('Surface_Tp', items)

get_int_feat('F_F_Class', items)

7845
************************************

signal..............
1:0.434131736527 out of 2004
0:0.15391200137 out of 5841

lanes..............
1:0.0254777070064 out of 157
0:0.0132450331126 out of 151
3:0.467075038285 out of 653
2:0.183505154639 out of 6305
5:0.618421052632 out of 76
4:0.505030181087 out of 497
6:0.5 out of 6

width..............
9:0.103196859226 out of 1783
19:0.290178571429 out of 1792
29:0.447961373391 out of 1864
14:0.0955943474647 out of 2406

hwy_type..............
0:0.40490797546 out of 163
1:0.131540546175 out of 4797
2:0.509803921569 out of 204
3:0.386850152905 out of 1962
4:0.0526315789474 out of 19
5:0.238938053097 out of 113
6:0.354430379747 out of 79
7:0.205882352941 out of 102
8:0.286821705426 out of 129
9:0.292857142857 out of 140
10:0.4 out of 40
11:0.377358490566 out of 53
12:0.555555555556 out of 9
13:0.0 out of 6
14:0.5 out of 6
15:0.6 out of 5
16:0.2 out of 5
17:1.0 out of 2
18:0.5 out of 2
19:0.0 out of 2
20:0.0 out of 2
22:0.0 out of 1
23:0.75 out 

KeyError: 'SPEEDLIMIT'

In [ ]:
def get_non_int_feat(featname, items):
    results = {}
    for x in non_inters:
        value = x['properties'][featname]
        if value not in results.keys():
            results[value] = {'crash': 0, 'no_crash': 0}
        if x['id'] in crashes_by_id.keys():
            results[value]['crash'] += 1
        else:
            results[value]['no_crash'] += 1
            
    print "\n" + featname + ".............."
    for key, value in results.iteritems():
        print str(key) + ':' + str(float(value['crash'])/(float(value['no_crash'] + value['crash']))) + " out of " + str(value['no_crash'] + value['crash'])


In [ ]:
non_inters = fiona.open(MAP_FP + 'non_inters_segments.shp')
print len(non_inters)
print non_inters[0]
count = 0

# Feats = ['hwy_type', 'width', 'lanes', 'osm_speed', 'signal', 'SPEEDLIMIT', 'Struct_Cnd', 'Surface_Tp', 'F_F_Class']

lanes = {}
for x in non_inters:
        
    num_lanes = x['properties']['lanes']
    
    if num_lanes not in lanes.keys():
        lanes[num_lanes] = {'crash': 0, 'no_crash': 0}
    if x['id'] in crashes_by_id.keys():
        count += 1
        lanes[num_lanes]['crash'] += 1
    else:
        lanes[num_lanes]['no_crash'] += 1
    

print "Crash rate for non-intersections:" + str(float(count)/float(count + len(non_inters)))
print lanes

get_non_int_feat('lanes', non_inters)

get_non_int_feat('signal', non_inters)
get_non_int_feat('hwy_type', non_inters)
get_non_int_feat('SPEEDLIMIT', non_inters)
get_non_int_feat('Struct_Cnd', non_inters)
get_non_int_feat('Surface_Tp', non_inters)
get_non_int_feat('F_F_Class', non_inters)